In [ ]:
import requests
import json

This notebook provides a deeper dive into the TRAPI options for querying the ROBOKOP KG using the ARA functionality. It assumes you are familiar with the concepts covered in `HelloRobokop.ipynb` and `HelloRobokop_TRAPI.ipynb`.  This follows the same format and process as in `HelloRobokop_TRAPI.ipynb` except using the API to Aragorn and examining returned scoring information.

TRAPI Documentation: https://github.com/NCATSTranslator/ReasonerAPI

Most TRAPI documents contain a `message` key.  Within that `message` are a `query_graph` denoting the user query,
a `knowledge_graph` consisting of the union of all nodes and edges that match the `query_graph` pattern, and a list of `results` that bind `query_graph` elements to `knowledge_graph` elements.

The following message contains only a `query_graph`.  This query graph consists of 3 nodes connected together in a line.   Two of the nodes (`n00` and `n02`) have specified identifiers, while the middle node of the line does not.  Rather the middle node has a list of `categories` that are acceptable.

For a researcher who is starting from a `name` who wants to use TRAPI, they can use the Name Resolver tool to get the identifers for the nodes.  This is illustrated in the `HelloRobokop_TRAPI_multiple_IDs.ipynb` notebook. 

This query asks "Find me a Biological Process or Activity, or a Gene, or a Pathway that is related to both `PUBCHEM.COMPOUND:644073` (Buprenorphine) and `HP:0001337` (Tremor).

In [ ]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ["PUBCHEM.COMPOUND:644073"],
            "categories": ["biolink:ChemicalEntity"]
          },
          "n01": {
              "categories": ["biolink:BiologicalProcessOrActivity","biolink:Gene","biolink:Pathway"]
          },
          "n02": {
            "ids": ["HP:0001337"],
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


This query can be sent to various components of Translator as needed.  It can be sent directly to the ROBOKOP KG hosted in the Automat system like this:

In [ ]:
aragorn_submit = "https://aragorn-u24.apps.renci.org/robokop/query"
response = requests.post(aragorn_submit,json=query)
print(response.status_code)
number_pathway_results = len(response.json()['message']['results'])
print(len(response.json()['message']['results']))

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=5)

The response in JSON form is a python dictionary with three main keys, the `message`, `log_level`, and `workflow`.  The `message` property contains the `query_graph`, `knowledge_graph`, and `results` from the query.

In [ ]:
print(response.json().keys())
print(response.json()['message'].keys())

query_out = response.json()['message']['query_graph']
kg = response.json()['message']['knowledge_graph']
results = response.json()['message']['results']

The original, submitted query graph with additional fields is returned with the `message` property in the `query_graph` property.

The code below shows the changes between the submitted query graph and the results query graph. The results includes additional attributes such as `knowledge_type`, `attribute_constraints`, and `qualifier_constraints`.  These attributes can be explicitly defined when submitting to automat such that further filtering can be done.  Because nothing was specified for these attributes in the submitted query graph, their values in the returned query graph are blank.

In [ ]:
edges = ["e00", "e01"]
nodes = ["n00", "n01", "n02"]

print(nodes)
print(edges)
pp.pprint(query)
pp.pprint(query_out)

The `results` property contains pathways resulting from the query message. Each pathway is organized into edge_bindings and node bindings and contains results for the edges and nodes specified in the query message.  Results are defined using the node and edge identifiers. The attributes for those nodes and edges (including the names) are available via the `knowledge_graph` component of the `message` section of the response.

In [ ]:
pp.pprint(results)

The `knowledge_graph` contains information about each of the Nodes and Edges found in `results`.  An example of a Node and an Edge are shown below.  

In [ ]:
pp.pprint(kg.keys())

Information returned for the each Node includes the concept ID (key), biolink categories, the name/label, attributes, the value type, and others.  Note that each entry under the `nodes` level is itemized in dictionary format with the key corresponding to the identifier used to define the `Result`.  The content for one Node is shown below.

In [ ]:
next(iter( kg['nodes'].items() ))

Information returned for the each Edge includes the edge ID (key), the subject's concept ID, the object's concept ID, the edge's predicate, any qualifiers, and attributes.  Note that each entry under the `edges` level is itemized in dictionary format  with the key corresponding to the identifier used to define the `Result`.  The content for one Edge is shown below.

In [ ]:
next(iter( kg['edges'].items() ))

The following sections show how the `knowledge_graph` can be used to annotate the `results`, which serve to bind the components of the `knowedge_graph` to the original `query_graph` used to find the `knowledge_graph`. A single result is shown below to highlight the features we'll use for the subsequent summaries. The `node_bindings` link the components of the `query_graph` to the identifiers of the nodes within the `knowledge_graph`. The keys for these bindings correspond to the identifiers used to define the `query_graph` above. For this example nodes are defined with an "n" followed by a number corresponding to the order of the node in the `query_graph`. The `edge_bindings` have identifiers defined with "e" plus a number specifying the order. In both cases, the `id` field corresponds to the corresponding identifier in the `knowledge_graph`. The `result` includes a score field defined with "s" plus a number, which is not used when directly querying ROBOKOP using TRAPI. This field is present here due to using the Aragorn API.

In [ ]:
# Illustrating the structure of each pathway result from the message property
pp.pprint(results[0])

To export the results from our query, we create a directory with today's date to hold the output files and then convert our output dictionaries into a set of tables, which can be easily exported.

In [ ]:
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
write_dir = Path("output/TRAPI",str(dt_string))
write_dir.mkdir(parents=True, exist_ok=True)

The code below writes out nodes from our `results`, but NOT the edges connecting the nodes.

In [ ]:
import pandas as pd
import os

cols = []
for node in sorted(results[0]['node_bindings'].keys()):
    cols.append(node)
    cols.append(node + '_name')
results_df = pd.DataFrame(columns = cols)

results_list = []
for result in results:
    result_dict = {}
    for node in sorted(result['node_bindings'].keys()):
        node_id = result['node_bindings'][node][0]['id']
        result_dict[node] = node_id
        result_dict[node + '_name'] = kg['nodes'][node_id]['name']

    results_list.append(pd.DataFrame([result_dict]))
results_df = pd.concat(results_list)
display(results_df)
results_df.to_csv(os.path.join(write_dir,'results_TRAPI.csv'), index=False)

combined_node_list = ["_".join([row[1].replace(" ", "_"), row[3].replace(" ", "_"), row[5].replace(" ", "_")]) for row in results_df[cols].to_numpy()]
pp.pprint(combined_node_list)

The edge IDs are then retrieved from the `results` and used to find the corresponding `predicate` label in the `knowledge_graph`. Two nodes can have more than one edge connecting them because each edge represents a distinct type of association derived from a single data source. Because of this, the number of rows in this report may differ from the set of results above. For an answer with three nodes such as this one, there will be at least one edge between nodes 1 and 2 as well as nodes 2 and 3. The direction of the edge connecting two nodes can differ as well, such as for "Buprenorphine" and "CYP2D6" below. Sometimes a single association type will be derived from multiple data sources. In the code below, we print a single edge for each association type and include the count of the number of data sources where that association was found. The following writes out all unique edges for each of the results in the format of `subject` -> `predicate` -> `object` and exports the information into a single file per `result`.

In [ ]:
from collections import Counter
import json
import pprint
pp = pprint.PrettyPrinter(indent=5)

for i in range(number_pathway_results):
    print(f"Pathway result: {combined_node_list[i]}")
    edge_bindings = results[i]['edge_bindings']

    edge_ids = []
    for edge_name, edge_list in edge_bindings.items():
        edge_ids.append({edge_name: [x['id'] for x in edge_list]})

    string_out_list = []
    for edge_dict in edge_ids:
        for edge_name, edge_list in edge_dict.items():
            for edge_id in edge_list:
                subject_id = kg['edges'][edge_id]['subject']
                subject = kg['nodes'][subject_id]['name']
                predicate = kg['edges'][edge_id]['predicate']
                object_id = kg['edges'][edge_id]['object']
                object = kg['nodes'][object_id]['name']
                string_out = f"{subject} -> {predicate} -> {object}"
                string_out_list.append(string_out)
    string_out_dict = dict(Counter(string_out_list).items())
    pp.pprint(string_out_dict)
    print("")
    
    with open(os.path.join(write_dir,combined_node_list[i]+".txt"), 'w') as convert_file:
        convert_file.write(json.dumps(string_out_dict))
        

We see that the connection between Buprenorphine and CYP2D6 consists of two edge types `biolink:affects` and `biolink:directly_physically_interacts_with`. The `biolink:affects` relationship is bidirectional with one edge starting with Buprenorphine (`Buprenorphine -> biolink:affects -> CYP2D6`) and one starting with CYP2D6 (`CYP2D6 -> biolink:affects -> Buprenorphine`). These three edges are repeated for all results that include Buprenorphine and CYP2D6 as the first two nodes. Those results differ in the type of tremor found for the third node. There are three edges connecting Buprenorphine and OPRM1 (`biolink:affects`, `biolink:directly_physically_interacts_with`, `biolink:related_to`). Of note, the `biolink:affects` was identified in 4 different knowledge sources as indicated by the count reported after the colon.

In [ ]:
pp.pprint(results[0].keys())
pp.pprint(results[0]['node_bindings'].keys())
pp.pprint(results[0]['edge_bindings'].keys())
pp.pprint(results[0]['score'])

In [ ]:
for result in results:
    print(result['score'])

In [ ]:
aragorn_result_summaries = []
for r in results:
    rs = f"Score={round(r['score'], 3)}: "
    j = 0
    while j < len(nodes):
        node_id = r['node_bindings'][nodes[j]][0]['id']
        node_name = kg['nodes'][node_id]['name']
        rs = rs + f"{node_name} ({node_id})"
        if j < len(edges):
            edge_id = r['edge_bindings'][edges[j]][0]['id']
            edge_name = kg['edges'][edge_id]['predicate']
            rs = rs + f"--{edge_name}-->"
        j = j + 1
    aragorn_result_summaries.append(rs)

In [ ]:
for rs in aragorn_result_summaries:
    print(rs)

The following assumes that the node names will sort in the correct order, which is the case with the default naming conventions. This exports the results showing the nodes and the score assigned to each result.

In [ ]:
cols = []
for node in sorted(results[0]['node_bindings'].keys()):
    cols.append(node)
    cols.append(node + '_name')
results_df = pd.DataFrame(columns = cols)

for result in results:
    result_dict = {}
    for node in result['node_bindings'].keys():
        node_id = result['node_bindings'][node][0]['id']
        result_dict[node] = node_id
        result_dict[node + '_name'] = kg['nodes'][node_id]['name']
    result_dict['score'] = result['score']
    #print(result_dict)

    results_df = results_df.append(result_dict, ignore_index=True)
print(results_df)
results_df.to_csv(os.path.join(write_dir,'results_TRAPI_aragorn.csv'))


In [ ]:
pp.pprint(kg['edges'])